# CE 295 Final Project
# EV Charging Schedule Optimization

# Optimization Solver

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from cvxpy import *

## Import Oakland K Feeder 1103 Existing Load Profile
### (pre-processed in Oakland_K_1103_Load.ipynb)

Source:
PG&E Integration Capacity Analysis (ICA) Map: https://www.pge.com/b2b/distribution-resource-planning/integration-capacity-map.shtml
Download Spatial Data --> Open GDB in QGIS --> Open Attribute Table for FeederLoadProfile--> Filter for Oakland K Substation Feeder 1103

In [2]:
oak_load = (pd.read_csv("OAK1103_Hrly_Load.csv"))
oak_load = oak_load.drop(columns = ['Unnamed: 0'])
oak_load

,Hour,Low_kW,Med_kW,High_kW
0,0,2222.0,2377.0,2531.0
1,1,2238.0,2386.0,2535.0
2,2,2092.0,2230.0,2368.0
3,3,1940.0,2076.0,2212.0
4,4,1844.0,1972.0,2100.0
5,5,1800.0,1927.0,2053.0
6,6,1819.0,1958.0,2097.0
7,7,1908.0,2111.0,2313.0
8,8,2077.0,2380.0,2683.0
9,9,2066.0,2366.0,2665.0


## Define Oakland K Feeder 1103 Capacity

Source:
PG&E Distribution Investment Deferral Framework (DIDF) Map: https://www.pge.com/b2b/distribution-resource-planning/grid-needs-assessment-map.html
Download Data --> PGE_2023_GNA_Appendix_E_Public.xlsx --> Filter for Oakland K 1103 (Bank and Feeder Capacity Needs)

In [3]:
oak_capacity = 7.36 * 1000 # kW

## Import Example Synthetic Dataset for EV Specs + Driver Behavior
### (pre-processed in Synthetic EV Data.ipynb)

“Tesla Model 3 Long Range Dual Motor,” EV Database. https://ev-database.org/car/1992/Tesla-Model-3-Long-Range-Dual-Motor

“Onboard Charger | Tesla Support,” Tesla, 2024. https://www.tesla.com/support/charging/onboard-charger (accessed Apr. 04, 2024)

In [4]:
veh_pop_full = pd.read_csv("Vehicle_Input_Data.csv")
veh_pop_full = veh_pop_full.drop(columns = ['Unnamed: 0'])
veh_pop_full

,VehID,Make,Model,Battery_Capacity_kWh,Charging_Rating_kW,Fuel_Econ_kWh/mi,Init_SOC,AM_Commute_Hr,PM_Commute_Hr,Commute_mi_per_trip,charge_access
0,1,Tesla,Model 3 Long Range,75,11,0.24,0.39,7,16,20,home
1,2,Tesla,Model 3 Long Range,75,11,0.24,0.70,8,17,20,both
2,3,Tesla,Model 3 Long Range,75,11,0.24,0.59,7,17,20,home
3,4,Tesla,Model 3 Long Range,75,11,0.24,0.32,6,18,20,work
4,5,Tesla,Model 3 Long Range,75,11,0.24,0.46,6,17,20,work
5,6,Tesla,Model 3 Long Range,75,11,0.24,0.80,7,16,20,home
6,7,Tesla,Model 3 Long Range,75,11,0.24,0.19,6,17,20,home
7,8,Tesla,Model 3 Long Range,75,11,0.24,0.49,6,17,20,both
8,9,Tesla,Model 3 Long Range,75,11,0.24,0.74,7,16,20,work
9,10,Tesla,Model 3 Long Range,75,11,0.24,0.42,8,18,20,both


## For First Test of Code, Use Just One Vehicle

In [5]:
veh_num = 1

In [6]:
veh_pop = veh_pop_full[0:veh_num]
veh_pop

,VehID,Make,Model,Battery_Capacity_kWh,Charging_Rating_kW,Fuel_Econ_kWh/mi,Init_SOC,AM_Commute_Hr,PM_Commute_Hr,Commute_mi_per_trip,charge_access
0,1,Tesla,Model 3 Long Range,75,11,0.24,0.39,7,16,20,home


## Prepare Timeseries Dataframes for known information

### Driving Energy Consumption for each Hour and Vehicle (veh_num x 24hr array) -- populate based on commuting hours, fuel economy, and trip length

In [7]:
driving_df = pd.DataFrame(columns = ["VehID"] + ["Hr"+str(i) for i in range(24)])
driving_df["VehID"] = veh_pop["VehID"]

# Fill in all Hr columns with 0s to start
for col in driving_df.columns[1:]:
    driving_df[col] = [0]*veh_num

# Fill in commuting behavior
for row in range(veh_num):
    
    # Update value for AM commute hour
    am_hr = veh_pop["AM_Commute_Hr"][row]
    driving_df["Hr" + str(am_hr)][row] = round(veh_pop["Fuel_Econ_kWh/mi"][row] * veh_pop["Commute_mi_per_trip"][row],2)
    
    # Update value for PM commute hour
    pm_hr = veh_pop["PM_Commute_Hr"][row]
    driving_df["Hr" + str(pm_hr)][row] = round(veh_pop["Fuel_Econ_kWh/mi"][row] * veh_pop["Commute_mi_per_trip"][row],2)

driving_df

/var/folders/66/5w9ddbys20s46gg6n2pv35r00000gn/T/ipykernel_19949/1606175932.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  driving_df["Hr" + str(pm_hr)][row] = round(veh_pop["Fuel_Econ_kWh/mi"][row] * veh_pop["Commute_mi_per_trip"][row],2)


,VehID,Hr0,Hr1,Hr2,Hr3,Hr4,Hr5,Hr6,Hr7,Hr8,...,Hr14,Hr15,Hr16,Hr17,Hr18,Hr19,Hr20,Hr21,Hr22,Hr23
0,1,0,0,0,0,0,0,0,4.8,0,...,0,0,4.8,0,0,0,0,0,0,0


### Convert to Array

In [8]:
driving_array = np.array(driving_df.drop(columns=["VehID"]))
driving_array.shape

(1, 24)

### Charger Availability (yes = 1 or no = 0) for each hour and vehicle (veh_num x 24hr array) -- populate based on charger access (home/work/both) and commuting schedule

In [9]:
chgr_avail_df = pd.DataFrame(columns = ["VehID"] + ["Hr"+str(i) for i in range(24)])
chgr_avail_df["VehID"] = veh_pop["VehID"]

# Iterate by vehicle
for row in range(veh_num):
    
    # Pull charge_access classification
    chgr_access = veh_pop["charge_access"][row]
    
    # Fill in 0's and 1's
    if chgr_access == "both":
        # charger always available
        chgr_avail_df.loc[row,chgr_avail_df.columns[1:]] = [1]*24
    
    elif chgr_access == "home":
        # charger available for hours before AM commute and after PM commute
        before_AM = [1 for hr in range(0,veh_pop["AM_Commute_Hr"][row])] # yes charger
        at_work = [0 for hr in range(veh_pop["AM_Commute_Hr"][row], veh_pop["PM_Commute_Hr"][row]+1)] # no charger
        after_PM = [1 for hr in range(veh_pop["PM_Commute_Hr"][row] + 1, 24)] # yes charger
        chgr_avail_df.loc[row,chgr_avail_df.columns[1:]] = before_AM + at_work + after_PM
    
    else: # "work"
        # charge available after AM commute and before PM commute
        before_AM = [0 for hr in range(0,veh_pop["AM_Commute_Hr"][row] + 1)] # no charger
        at_work = [1 for hr in range(veh_pop["AM_Commute_Hr"][row] + 1, veh_pop["PM_Commute_Hr"][row])] # yes charger
        after_PM = [0 for hr in range(veh_pop["PM_Commute_Hr"][row], 24)] # no charger
        chgr_avail_df.loc[row,chgr_avail_df.columns[1:]] = before_AM + at_work + after_PM
        
chgr_avail_df

,VehID,Hr0,Hr1,Hr2,Hr3,Hr4,Hr5,Hr6,Hr7,Hr8,...,Hr14,Hr15,Hr16,Hr17,Hr18,Hr19,Hr20,Hr21,Hr22,Hr23
0,1,1,1,1,1,1,1,1,0,0,...,0,0,0,1,1,1,1,1,1,1


### Convert to array

In [10]:
chgr_avail_array = np.array(chgr_avail_df.drop(columns = ["VehID"]))
chgr_avail_array.shape

(1, 24)

## Optimization Problem Set-Up

Code Outline:

* Hr0 --> we know initial SOC at the start for each vehicle
    * Define charge used for driving (should update the matrix to multiply through by kWh/mi * mi since assuming all the same -- can build into veh_pop if want)
       * Known parameter from imported EV info, TBD if needs to be declared as constraints
    * Define charge added to battery as (charging rate fraction) * (max charging rate) * (duration)
        * Charging Rate Fraction should be a veh_num x 24 hrs array of variables
            * Constraint: 0 <= r <= 1
        * Max charging rate is a known parameter (vals imported as part of EV info)
        * Duration is set at 1 hour until decide to update the code to change intervals
    * Calculate the SOC at the end of the hour = SOC for start of next Hr
        * SOC(t+1) = ((SOC(t) * Battery Capacity kWh) - Driving kWh + Charging kWh)/Battery Capacity kWh
        * SOC should be set up as a veh_num x 24 hrs array of variables and as part of the constraints step, set the initial condition for SOC at hr0 for each vehicle
        * SOC cannot fall below 10%
        * SOC must be at least 80% by the time done charging
            * need to find a way to indicate start and end of charging period
            * and try to incorporate constraint that makes it continuous once charging starts
* Iterate over all timesteps 
* At the end should have a filled out array of charging rate fractions by vehicle and hr
* Can multiply it by the max charging rate (tech. spec) for each vehicle
* Then sum up EV charging load across all vehicles for each hour
    * Need to represent this calculation as a constraint and EV charging load by hour as var
* Then sum the existing load and the EV charging load to get total load by hour
    * the existing load is a known dataset
    * this total load is constrained by the Feeder capacity
    * it is also constrained by a ramp rate limit --> make an assumption of <20% for now
    * The array of total load by hour is the basis for the optimization fxn goal


### Identify Existing Load Data that want to use (have low, medium, and high estimates available --- starting with medium and can see what have time for)

In [11]:
exist_load = np.array([oak_load["Med_kW"]]).reshape(24)
exist_load

array([2377., 2386., 2230., 2076., 1972., 1927., 1958., 2111., 2380.,
       2366., 2113., 1802., 1500., 1247., 1197., 1208., 1357., 1732.,
       2307., 2776., 2973., 3051., 2979., 2700.])

### Define Optimization Variables

In [12]:
time_steps = 24 # hours
time_delta = 1 # hour

# Total Load = Existing Load + EV Load (by hour)
tot_load = Variable(time_steps)

# EV Load = Sum EV Load over all vehicles (by hour)
ev_load = Variable(time_steps)

# Charge Added (Load) (kWh), by vehicle, by hour
chrg_added = Variable((veh_num, time_steps))

# Charging Rate as Fraction of Maximum Feasible Rate (by vehicle, by hour)
chrg_rate_frac = Variable((veh_num, time_steps))

# Battery State of Charge (SOC) at start of each hour (by vehicle, by hour)
soc = Variable((veh_num, time_steps))


### Define Objective Function

In [13]:
# Minimize the peak demand once EV incorporated

objective = Minimize(max(tot_load))

### Define Constraints

In [14]:
constraints = []

#### Specify Initial Conditions for SOC as equality constraints

In [15]:
constraints += [soc[veh,0] == veh_pop["Init_SOC"][veh] for veh in range(veh_num)]

#### Constrain SOC to be between 0 and 1

In [16]:
constraints += [0 <= soc]
constraints += [soc <= 1]

#### Constraint SOC to specify that at least 80% charging must be reached by a certain time (TBD - NOT IMPLEMENTED YET)

In [17]:
# TBD




#### Constrain Charging Rate Fraction:
1) 0 <= r <= 1 (by definition as a fraction of max value)
2) r <= Charger Availability Status --> this ensures that the charging rate goes to 0 if there is no charger available (status = 0) --> if there is a charger available, it acts as a redundant constraint on <= 1

In [18]:
constraints += [0 <= chrg_rate_frac]
constraints += [chrg_rate_frac <= 1]
constraints += [chrg_rate_frac <= chgr_avail_array]

#### Constrain Battery Dynamics

In [19]:
# Generate Constraints by iterating over all time steps

# Iterate by Vehicle
for veh in range(veh_num):
    
    # hrs 0 to 23 -- may be refined to more granular time intervals time permitting
    for t in range(time_steps):
    
        # Charge Supply from EV Charging = Charging Rate Fraction * Max Rate * Duration
        # Charging Rate Fraction is a set of optimization variables to be determined
        constraints += [chrg_added[veh,t] == chrg_rate_frac[veh,t] * veh_pop["Charging_Rating_kW"][veh] * time_delta]

        # State of Charge at End of Time Step / For Start of Next Time Step
        # kWh @ start of hour + kWh added by charging - kWh consumed by driving
        # charging OR driving OR no action for any given timestep (accounted for in setup)
        # Renormalized back to SOC fraction

        # If on the last time step index, no need to calculate the next one
        if t == time_steps - 1:
            pass
        else:
            constraints += [soc[veh,t+1] == ( (soc[veh,t] * veh_pop["Battery_Capacity_kWh"]) + chrg_added[veh,t] - driving_array[veh,t] ) / veh_pop["Charging_Rating_kW"][veh]]



#### Constrain EV Load (equality constraint summing charg_added across vehicles by time step)

In [20]:
for time in range(time_steps):
    
    constraints += [ev_load[time] == chrg_added[:,time].cumsum(axis = 0)]

#### Constrain Total Load (equality constraint at each time step)

In [21]:
for time in range(time_steps):
    
    constraints += [tot_load[time] == ev_load[time] + exist_load[time]]

#### Constrain Total Load to remain below kW capacity limit of Feeder

In [22]:
constraints += [tot_load <= oak_capacity]

#### Constrain Total Load Ramp Rates between time steps

In [24]:
# Assume a ramp limit --> The load at current time step cannot be more than X% of prior load
ramp_limit = 0.10

for time in range(1, time_steps):
    constraints += [(tot_load[time] * 1/tot_load[time-1] - 1) <= ramp_limit]

### Run Optimization Solver

In [25]:
one_car = Problem(objective, constraints)
one_car.solve()

DCPError: Problem does not follow DCP rules. Specifically:
The following constraints are not DCP:
var1[1] @ 1.0 / var1[0] + -1.0 <= 0.1 , because the following subexpressions are not:
|--  var1[1] @ 1.0 / var1[0]
var1[2] @ 1.0 / var1[1] + -1.0 <= 0.1 , because the following subexpressions are not:
|--  var1[2] @ 1.0 / var1[1]
var1[3] @ 1.0 / var1[2] + -1.0 <= 0.1 , because the following subexpressions are not:
|--  var1[3] @ 1.0 / var1[2]
var1[4] @ 1.0 / var1[3] + -1.0 <= 0.1 , because the following subexpressions are not:
|--  var1[4] @ 1.0 / var1[3]
var1[5] @ 1.0 / var1[4] + -1.0 <= 0.1 , because the following subexpressions are not:
|--  var1[5] @ 1.0 / var1[4]
var1[6] @ 1.0 / var1[5] + -1.0 <= 0.1 , because the following subexpressions are not:
|--  var1[6] @ 1.0 / var1[5]
var1[7] @ 1.0 / var1[6] + -1.0 <= 0.1 , because the following subexpressions are not:
|--  var1[7] @ 1.0 / var1[6]
var1[8] @ 1.0 / var1[7] + -1.0 <= 0.1 , because the following subexpressions are not:
|--  var1[8